## Data Preprocessing

In [1]:
# Importing Necessary Packages
import warnings
warnings.filterwarnings("ignore")

import time
from datetime import date, timedelta, datetime
import numpy as np
import pandas as pd
import statistics as st
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import datetime
from pandas import Series,DataFrame

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

## Setting Logging

In [2]:
import logging
import logging.handlers
import os
 
handler = logging.handlers.WatchedFileHandler(
    os.environ.get("LOGFILE", "../logs/prediction.log"))
formatter = logging.Formatter(logging.BASIC_FORMAT)
handler.setFormatter(formatter)
root = logging.getLogger()
root.setLevel(os.environ.get("LOGLEVEL", "INFO"))
root.addHandler(handler)
logging.info("Testing Loggings") 
try:
    exit(main())
except Exception:
    logging.exception("Exception in main()")
    exit(1)

## Creating Dataframe Class, and object

In [3]:
# Class to load the data
class FetchData():
    def __init__(self):
        self.dfdict = {}
        self.dfdict['train'] = self.get_train_data()
        self.dfdict['test'] = self.get_test_data()
        self.dfdict['sample'] = self.get_sample_data()
        self.dfdict['store'] = self.get_store_data()
        
    def get_train_data(self,name='train'):
        filename = f'../data/{name}.csv'
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")
            return df

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None
    
    def get_test_data(self,name='test'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")
            return df   

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None
        
    def get_store_data(self,name='store'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully") 
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None 
    
    def get_sample_data(self,name='sample_submission'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")  
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None 
    
    def show_sample_data(self,dataset='train', sample=5):
        try:
            df = dataset.head(sample)
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in getting sample data of a dataset, {e}")
            return None

    def find_null_values(self,dataset='train'):
        try:
            df = dataset.isnull().sum()
            logging.info("Getting Null values, Execution successfuly")
            return df  

        except Exception as e:
            logging.exception(f" Exception in getting Null values, {e}")
            return None

    def get_start_end_date(self, dataset_name='train', date_column='Date'):
        try:
            start_date = dataset_name[date_column].min()
            end_date = dataset_name[date_column].max()
            logging.info("Getting start and End date successfully")
            return start_date, end_date

        except Exception as e:
            logging.exception(f"Exception in getting start and end date, {e}")

            return None, None

    def join_dataset(self, dataset='train', dataset1='store'):
        if dataset.Store.nunique() == dataset1.Store.nunique():
            try:
                df_combined = dataset.merge(dataset1, how='left', left_on=dataset.Store, right_on=dataset1.Store)
                df_combined.drop(['key_0', 'Store_y'], axis=1, inplace=True)
                df_combined = df_combined.rename(columns={'Store_x':'Store'})
                logging.info(f" Joining {dataset} and {dataset1} datasets successfully")

                return df_combined.shape, df_combined
            
            except Exception as e:
                logging.debug(f"Exception in Joining {dataset} and {dataset1} datasets, {e}")
                return None, None

        else:
            logging.error("The values in the dataset are not compartible")
            print("The values in the dataset are not compartible")

    def add_day_month_year_to_dataset(self, dataset, column_list = ['day','month','year']):
        try:
            dataset.Date = pd.to_datetime(dataset.Date)
            for column in column_list:
                dataset[column] = dataset.Date.dt.column
                logging.info(f"Adding {column} column to dataset successfully")
                return dataset

        except Exception as e:
            logging.exception(f"Exception occured in Adding columns in dataset, Exception:{e}")
             
            return None

In [4]:
#  Creating FetchData Object
data = FetchData()

### Loading Datasets

In [ ]:
train_data = data.get_train_data("train")
store_data = data.get_train_data("store")
test_data = data.get_train_data("test")
sample_submission_data = data.get_train_data("sample_submission")